### Vorgehen:
1. "unwichtige" Parameter herausfinden durch Experimente mit num_exp=1 und reduce_dataset=0.2.
    Parameter: Backbone, kd(schhonmal testen)
2. Wie 1. aber mit num_exp=3.
    Parameter: classification, selection_method
- Dadurch schonmal grobe Infos sammeln und dann später stichprobenartig verifizieren

3. Mit den gesammelten Infos die Benchmarks übertreffen oder Schonmal Q2 und Q3 anfangen

## ---

### Sweep fj01kgs5: sc_q1_all_datasets_classification_and_selection
- reduce_dataset=1.1 statt 0.2:   
  
|        | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet |
|--------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------|
| Selection| bayesian | bayesian | bayesian | bayesian | bayesian | bayesian |      | bayesian | bayesian |
| Classification  | inv_eucld/random/ws | ws/kl/eucl | ??? | ws/kl/eucl | ws/random/inv_ws/inv_eucld | random * |      | ws/random/kl/eucld | inv_eucld |  

\* Alles anderer ca. gleich gut  

- Dass die Sachen ähnlich gut performen ist nicht schlimm: geringe Daten und Task Menge.
- Aber warum ist random manchmal so gut wie die anderen?
- Die lr war viel zu hoch deswegen erstmal nochmal mit geringerer lr versuchenund dann gucken 1. ob die Ergebnisse besser sind, 2. wenn ja dann sind die versuche besser (weniger random) und haben vor allem bei der Clasification mehr Aussagekraft!!
#### To Do
- Sweeps nochmal mit besserer lr durchführen
- Nochmal angucken wie die Distr. erstellt werden. Wenn die nach dem output genommen werden, gibt es bei moderater/hoher lr eine zu starke Veränedrung und es ist quasi wie random. Durch experimente bestätigen vor Mail
- Ein durchlauf mit gesammter datenmenge und tasks um unterschied zu sehen
- Kyra: kurz sweep beschreiben, Ergebnis: Bayesian ist besser, aber sie classification ist oft nah an random? Ist das schlimm, oder soll ich das einfach weiter untersuchen und dann mal gucken?

## ---

### Sweep 0kz93mu5: sc_q1_all_datasets_backbone_and_kd
- t

|               | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet |
|---------------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------|
| Backbone |          |           |     |              |           |      |      |              |                   |
| KD       |          |           |     |              |           |      |      |              |                   |
  
- bessere Parameter herausfinden damit nicht jede acc. 0.1 ist (tp49kob2)

## ---

### Sweep qxb0hd5i: sweep_config_batchsize_optimization
T=10; moe_max_experts=5; reduce_dataset=1
|        | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet | cddb |
|--------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------| ---- |
| GPU Memory| 7851.74 | 9655.29 | 7228.88 | 9655.29 | 6656.36 | 7486.83 | 7186.94 | 9xxx.xx | --- | --- |
  
-> letzen beiden checken  
-> implementation von batch_size *= 1.x

## ---

### cifar100
- backbone: